## Feature Engineering for Modeling 

### Steps to feature engineer: 
    1. create candidate features 
        * 4 week prior avgerage: points, yds, touches, touchdowns, fumbles, qbr 
        * opp win record, opp avg def pts allowed 
        * isStarter, isHurt 
    2. create incemental refresh schedule 
    3. write final data set to Google Drive

In [1]:
#importing various libraries 
import gspread 
#Service client credential from oauth2client
from oauth2client.service_account import ServiceAccountCredentials
# Print nicely
import pprint
#Create scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
#create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name('../quickstart/g_sheet_creds.json',scope)
#create gspread authorize using that credential
client = gspread.authorize(creds)
my_email = 'matthewjchristy66@gmail.com'

In [2]:
#data manipulation 
import pandas as pd 
import numpy as np

In [5]:
def read_file(sheet_name):
    out = client.open(sheet_name).sheet1
    out = out.get_all_values()
    out = pd.DataFrame(out, columns = out.pop(0))
    return(out)

In [6]:
#grabbing data for modeling 
passing = read_file(sheet_name = 'passing_processed_step1')
rush = read_file(sheet_name= 'rushing_processed_step1')
rec = read_file(sheet_name = 'receiving_processed_step1')

In [8]:
passing.head()

,Name,Week,Team,Opp,Score,Comp,Att,Yds,TD,Int,Sck,FUM,Rate,300yd_flag,passing_fantasy_pts
0,Andy Dalton,1,CIN,@ SEA,L 20-21,35,51,418,2,0,5,2,106.5,1,24.72
1,Dak Prescott,1,DAL,vs NYG,W 35-17,25,32,405,4,0,0,0,158.3,1,32.2
2,Matthew Stafford,1,DET,@ ARI,T 27-27,27,45,385,3,0,3,1,110,1,27.4
3,Case Keenum,1,WAS,@ PHI,L 27-32,30,44,380,3,0,1,0,117.6,1,27.2
4,Patrick Mahomes,1,KC,@ JAX,W 40-26,25,33,378,3,0,0,0,143.2,1,27.12


In [9]:
rush.head()

,Name,Week,Team,Opp,Score,Att,Yds,Avg,TD,FUM,100yd_flag,rush_fantasy_pts
0,Marlon Mack,1,IND,@ LAC,L 24-30,25,174,7.0,1,0,1,23.4
1,Christian McCaffrey,1,CAR,vs LA,L 27-30,19,128,6.7,2,0,1,24.8
2,Saquon Barkley,1,NYG,@ DAL,L 17-35,11,120,10.9,0,0,1,12
3,Dalvin Cook,1,MIN,vs ATL,W 28-12,21,111,5.3,2,0,1,23.1
4,Mark Ingram,1,BAL,@ MIA,W 59-10,14,107,7.6,2,0,1,22.7


In [10]:
rec.head()

,Name,Week,Team,Opp,Score,Rec,Yds,Avg,TD,FUM,100yd_flag,rec_fantasy_pts
0,Sammy Watkins,1,KC,@ JAX,W 40-26,9,198,22.0,3,0,1,37.8
1,Michael Gallup,1,DAL,vs NYG,W 35-17,7,158,22.6,0,0,1,15.8
2,John Ross,1,CIN,@ SEA,L 20-21,7,158,22.6,2,0,1,27.8
3,DeSean Jackson,1,PHI,vs WAS,W 32-27,8,154,19.2,2,0,1,27.4
4,Marquise Brown,1,BAL,@ MIA,W 59-10,4,147,36.8,2,0,1,26.7


In [ ]:
#creating a moving avg function 
def move_avg(df, smooth_col, group_vals, window):
    l_mean = lambda x: x.rolling(window, 1).mean()
    out = df.groupby(group_vals)[smooth_col].transform(l_mean)
    return(out)

In [15]:
#light cleaning 
pf = ['Comp', 'Att', 'Yds', 'TD', 'Int', 'Sck', 'FUM', 'Rate']
rrf = ['Yds', 'TD', 'FUM']
passing[pf] = passing[pf].astype(float)
rush[rrf] = rush[rrf].astype(float)
rec[rrf] = rec[rrf].astype(float)

In [22]:
out = rec.groupby('Name')['Yds'].transform(lambda x: x.rolling(4, 1).mean())

In [23]:
out.head()

0    198.0
1    158.0
2    158.0
3    154.0
4    147.0
Name: Yds, dtype: float64